This is the updated graph builder 

In [ ]:
# algorithms/graph_builder.py
import pandas as pd
from utils.geo_utils import haversine

def build_graph(node_csv_path):
    df = pd.read_csv(node_csv_path)
    print(f"🔍 Loaded {len(df)} nodes from {node_csv_path}")

    # build centroid lookup
    centroids = {
        str(r.Site_ID): (r.Latitude, r.Longitude)
        for r in df.itertuples()
    }

    # split intersections' “Location” by “/” into road names
    df['Roads'] = (
       df['Location']
         .str.split('/')
         .apply(lambda roads:[r.strip() for r in roads])
    )

    edges = []
    for road, grp in df.explode('Roads').groupby('Roads'):
        # order points along the road
        pts = (
            grp[['Site_ID','Latitude','Longitude']]
             .drop_duplicates()
        )
        # decide sort axis
        if pts['Latitude'].std() > pts['Longitude'].std():
            pts = pts.sort_values('Latitude', ascending=False)
        else:
            pts = pts.sort_values('Longitude')

        ids = [str(i) for i in pts['Site_ID']]
        for A, B in zip(ids, ids[1:]):
            latA, lonA = centroids[A]
            latB, lonB = centroids[B]
            d = haversine(latA,lonA,latB,lonB)
            # tag each directed edge with the road name
            edges.append({'A':A,'B':B,'distance':d,'road':road})
            edges.append({'A':B,'B':A,'distance':d,'road':road})

    print(f"🔗 Built {len(edges)} directed edges across all roads.")
    return centroids, edges


updated edge mapper 

In [ ]:
# utils/edge_mapper.py
import pandas as pd
import numpy as np
from math import radians, sin, cos, atan2, degrees

def bearing(lat1, lon1, lat2, lon2):
    φ1, φ2 = radians(lat1), radians(lat2)
    Δλ = radians(lon2 - lon1)
    x = sin(Δλ)*cos(φ2)
    y = cos(φ1)*sin(φ2) - sin(φ1)*cos(φ2)*cos(Δλ)
    θ = atan2(x, y)
    return (degrees(θ) + 360) % 360

class EdgeMapper:
    def __init__(self, volume_pkl):
        df = pd.read_pickle(volume_pkl)
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['Site_ID']   = df['Site_ID'].astype(str)

        # one arm per unique (Site,Location) with lat/lon
        arms = (
            df[['Site_ID','Location','Latitude','Longitude']]
              .drop_duplicates()
        )
        arms['Latitude']  = arms['Latitude'].astype(float)
        arms['Longitude'] = arms['Longitude'].astype(float)
        self.arms = arms.reset_index(drop=True)

    def best_arm(self, A, B, centroids, road=None):
        """
        Given directed edge A→B on a known `road`, pick the arm
        at site A whose Location best matches that road *and* bearing.
        """
        A, B = str(A), str(B)
        a_lat,a_lon = centroids[A]
        b_lat,b_lon = centroids[B]
        desired = bearing(a_lat, a_lon, b_lat, b_lon)

        cands = self.arms[self.arms['Site_ID']==A]
        if road:
            # only keep arms whose Location text mentions this road name
            mask = cands['Location'].str.contains(road, case=False, na=False)
            if mask.any():
                cands = cands[mask]
        if cands.empty:
            raise ValueError(f"No arms found for site {A} on road={road}")

        best_loc, best_diff = None, 360.0
        for _, row in cands.iterrows():
            arm_b = bearing(a_lat, a_lon, row['Latitude'], row['Longitude'])
            diff = abs((arm_b - desired + 180) % 360 - 180)
            if diff < best_diff:
                best_diff = diff
                best_loc  = row['Location']

        return best_loc


In [ ]:
wiring them together when building the weighted edges 

In [ ]:
# wherever you map flows onto edges
centroids, edges = build_graph(node_csv_path)
mapper = EdgeMapper(volume_pkl)

for e in edges:
    A, B, dist, road = e['A'], e['B'], e['distance'], e['road']
    arm_loc = mapper.best_arm(A, B, centroids, road=road)
    # now look up the traffic volume *for that exact* A/arm_loc combination
    vol = get_volume_for(A, arm_loc, desired_timestamp)
    speed = flow_to_speed(vol*4)
    ...
